In [1]:
#Loading the necessary packages!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, median_absolute_error
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import catboost as cb
from scipy.optimize import minimize

In [2]:
#Loasding the data!!
train = pd.read_csv("/kaggle/input/bagpack11/train.csv")
test = pd.read_csv("/kaggle/input/bagpack11/test.csv")
extra_train = pd.read_csv("/kaggle/input/bagpack11/training_extra.csv")


In [3]:
train.head(10)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
5,5,Nike,Canvas,Medium,10.0,No,Yes,NaN,Black,7.241812,20.01553
6,6,Nike,NaN,Large,3.0,No,No,Backpack,Green,6.828123,84.80500
7,7,Puma,Canvas,Small,1.0,Yes,Yes,Backpack,Blue,21.488864,27.15815
8,8,Under Armour,Polyester,Medium,8.0,Yes,No,Tote,Gray,10.207780,25.98652
9,9,Under Armour,Nylon,Medium,2.0,Yes,Yes,Messenger,Pink,15.895100,38.48741


In [4]:
test.head(10)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
1,300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
2,300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
3,300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
4,300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953
5,300005,Under Armour,Nylon,Medium,8.0,No,No,Backpack,Black,17.547673
6,300006,Nike,Nylon,Large,8.0,No,Yes,Tote,Blue,16.003025
7,300007,Jansport,Polyester,Medium,6.0,Yes,No,Backpack,Blue,24.238091
8,300008,Nike,Canvas,Large,8.0,Yes,Yes,Backpack,Red,19.181167
9,300009,Puma,Leather,Large,1.0,No,No,Tote,Blue,9.937962


In [5]:
extra_train.head(10)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,500000,Under Armour,Canvas,Small,10.0,Yes,Yes,Tote,Blue,23.882052,114.11068
1,500001,Puma,Polyester,Small,4.0,No,Yes,Backpack,Green,11.869095,129.74972
2,500002,Jansport,Polyester,Small,8.0,Yes,Yes,Tote,Red,8.092302,21.37370
3,500003,Nike,Nylon,Large,7.0,No,No,Messenger,Pink,7.719581,48.09209
4,500004,Nike,Leather,Large,9.0,No,Yes,Tote,Green,22.741826,77.32461
5,500005,Adidas,Polyester,Medium,5.0,No,No,Tote,NaN,18.834692,35.99599
6,500006,Jansport,Nylon,Large,5.0,Yes,Yes,Messenger,Blue,26.882904,40.81006
7,500007,Adidas,Polyester,Small,9.0,Yes,No,Tote,Gray,21.716360,71.12733
8,500008,Adidas,Leather,Small,6.0,Yes,Yes,Tote,Green,21.849587,76.50225
9,500009,Puma,Nylon,Medium,2.0,No,Yes,Tote,Red,19.059414,67.30516


In [6]:
#checking the number of rows and column present in the data

num_train_rows, num_train_columns = train.shape

num_test_rows, num_test_columns = test.shape

num_extra_rows, num_extra_columns = extra_train.shape

print("Training Data:")
print(f"Number of Rows: {num_train_rows}")
print(f"Number of Columns: {num_train_columns}\n")

print("Test Data:")
print(f"Number of Rows: {num_test_rows}")
print(f"Number of Columns: {num_test_columns}\n")

print("Extra Data:")
print(f"Number of Rows: {num_extra_rows}")
print(f"Number of Columns: {num_extra_columns}")

Training Data:
Number of Rows: 300000
Number of Columns: 11

Test Data:
Number of Rows: 200000
Number of Columns: 10

Extra Data:
Number of Rows: 3694318
Number of Columns: 11


This below code creates a summary table that displays information about missing values, unique values, and data types for features in multiple datasets (train, test, extra_train)

In [7]:
# Creating a table for missing values, unique values and data types of the features

missing_values_train = pd.DataFrame({'Feature': train.columns,
                              '[TRAIN] No. of Missing Values': train.isnull().sum().values,
                              '[TRAIN] % of Missing Values': ((train.isnull().sum().values)/len(train)*100)})

missing_values_test = pd.DataFrame({'Feature': test.columns,
                             '[TEST] No.of Missing Values': test.isnull().sum().values,
                             '[TEST] % of Missing Values': ((test.isnull().sum().values)/len(test)*100)})

missing_values_original = pd.DataFrame({'Feature': extra_train.columns,
                             '[ORIGINAL] No.of Missing Values': extra_train.isnull().sum().values,
                             '[ORIGINAL] % of Missing Values': ((extra_train.isnull().sum().values)/len(extra_train)*100)})

unique_values = pd.DataFrame({'Feature': train.columns,
                              'No. of Unique Values[FROM TRAIN]': train.nunique().values})
feature_types = pd.DataFrame({'Feature': train.columns,
                              'DataType': train.dtypes})

merged_df = pd.merge(missing_values_train, missing_values_test, on='Feature', how='left')
merged_df = pd.merge(merged_df, missing_values_original, on='Feature', how='left')
merged_df = pd.merge(merged_df, unique_values, on='Feature', how='left')
merged_df = pd.merge(merged_df, feature_types, on='Feature', how='left')

merged_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Feature,[TRAIN] No. of Missing Values,[TRAIN] % of Missing Values,[TEST] No.of Missing Values,[TEST] % of Missing Values,[ORIGINAL] No.of Missing Values,[ORIGINAL] % of Missing Values,No. of Unique Values[FROM TRAIN],DataType
0,id,0,0.000000,0.0,0.0000,0,0.000000,300000,int64
1,Brand,9705,3.235000,6227.0,3.1135,117053,3.168460,5,object
2,Material,8347,2.782333,5613.0,2.8065,102615,2.777644,4,object
3,Size,6595,2.198333,4381.0,2.1905,81190,2.197699,3,object
4,Compartments,0,0.000000,0.0,0.0000,0,0.000000,10,float64
5,Laptop Compartment,7444,2.481333,4962.0,2.4810,91089,2.465651,2,object
6,Waterproof,7050,2.350000,4811.0,2.4055,87274,2.362385,2,object
7,Style,7970,2.656667,5153.0,2.5765,96210,2.604270,3,object
8,Color,9950,3.316667,6785.0,3.3925,123667,3.347492,6,object
9,Weight Capacity (kg),138,0.046000,77.0,0.0385,1670,0.045205,181596,float64


In [8]:
#Count the duplicates in the train test and extra train 
train_duplicates = train.duplicated().sum()
test_duplicates = test.duplicated().sum()
extra_duplicates = extra_train.duplicated().sum()

print(f"Number of duplicate rows in train data: {train_duplicates}")
print(f"Number of duplicate rows in test data: {test_duplicates}")
print(f"Number of duplicate rows in extra train data: {extra_duplicates}")


Number of duplicate rows in train data: 0
Number of duplicate rows in test data: 0
Number of duplicate rows in extra train data: 0


In [9]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,300000.0,149999.500000,86602.684716,0.0,74999.750000,149999.500000,224999.250000,299999.0
Compartments,300000.0,5.443590,2.890766,1.0,3.000000,5.000000,8.000000,10.0
Weight Capacity (kg),299862.0,18.029994,6.966914,5.0,12.097867,18.068614,24.002375,30.0
Price,300000.0,81.411107,39.039340,15.0,47.384620,80.956120,115.018160,150.0


EDA

In [10]:
train = pd.concat([train, extra_train], ignore_index=True)


In [11]:
numerical_variables = ["Weight Capacity (kg)"]
target_variable = "Price"
categorical_variables = ['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment','Waterproof', 'Style', 'Color']

Data Imputation (Handling missing values)

Understanding Each Feature and Missing Data Handling
1️⃣ Brand (~3.24% missing in train, ~3.11% in test)
Unique Values: Jansport, Under Armour, Nike, Adidas, Puma, NaN
Imputation Strategy: Since it's categorical with only 5 unique brands, we can impute missing values with the mode (most frequent brand).
2️⃣ Material (~2.78% missing in train, ~2.81% in test)
Unique Values: Leather, Canvas, Nylon, Polyester, NaN
Imputation Strategy: Mode imputation works best here since materials are limited categories.
3️⃣ Size (~2.20% missing in train, ~2.19% in test)
Unique Values: Medium, Small, Large, NaN
Imputation Strategy: Since it's a well-defined categorical variable, we use mode imputation.
4️⃣ Compartments (✅ No missing values)
Unique Values: 10 unique numerical values
Imputation Strategy: ✅ No action needed.
5️⃣ Laptop Compartment (~2.48% missing in train, ~2.48% in test)
Unique Values: Yes, No, NaN
Imputation Strategy: Since it's a binary categorical variable, we use mode imputation.
6️⃣ Waterproof (~2.35% missing in train, ~2.41% in test)
Unique Values: Yes, No, NaN
Imputation Strategy: Mode imputation is best.
7️⃣ Style (~2.66% missing in train, ~2.58% in test)
Unique Values: Tote, Messenger, Backpack, NaN
Imputation Strategy: Mode imputation.
8️⃣ Color (~3.32% missing in train, ~3.39% in test)
Unique Values: Black, Green, Red, Blue, Gray, Pink, NaN
Imputation Strategy: Mode imputation.
9️⃣ Weight Capacity (kg) (~0.05% missing in train, ~0.04% in test)
Unique Values: Numeric
Imputation Strategy: Since it is a continuous numerical variable, we use median imputation to prevent extreme values from affecting the distribution.

In [12]:
# Define imputation strategies
categorical_features = ["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"]
numerical_features = ["Weight Capacity (kg)"]

# Fill categorical missing values with mode (most frequent value)
for col in categorical_features:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(test[col].mode()[0], inplace=True)

# Fill numerical missing values with median
for col in numerical_features:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(test[col].median(), inplace=True)

<ipython-input-12-cb2c177abb52>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace=True)
<ipython-input-12-cb2c177abb52>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [13]:
train.isnull().sum()

id                      0
Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

In [24]:
# Display all column names
print(X_train.columns)


Index(['id', 'Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)'],
      dtype='object')


In [25]:
# List of columns you want to check unique values for
columns_to_check = ['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
                    'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)']

# Loop through each column and print unique values
for col in columns_to_check:
    if col in X_train.columns:
        print(f"Unique values in '{col}':")
        print(X_train[col].unique())
        print("="*50)
    else:
        print(f"Column '{col}' not found in the dataset.")


Unique values in 'Brand':
['Under Armour' 'Adidas' 'Puma' 'Jansport' 'Nike']
Unique values in 'Material':
['Nylon' 'Polyester' 'Leather' 'Canvas']
Unique values in 'Size':
['Large' 'Medium' 'Small']
Unique values in 'Compartments':
[ 5.  7.  3.  6. 10.  4.  2.  1.  9.  8.]
Unique values in 'Laptop Compartment':
['Yes' 'No']
Unique values in 'Waterproof':
['Yes' 'No']
Unique values in 'Style':
['Tote' 'Backpack' 'Messenger']
Unique values in 'Color':
['Blue' 'Black' 'Green' 'Red' 'Gray' 'Pink']
Unique values in 'Weight Capacity (kg)':
[17.90855234 13.08145247 19.55185251 ... 23.50389097 16.025853
 16.60998863]


In [14]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
X = train.drop(columns='Price')  # Assuming 'Price' is the target
y = train['Price']




In [15]:
# Define categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns
print(categorical_cols)
print(numerical_cols)


Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')
Index(['id', 'Compartments', 'Weight Capacity (kg)'], dtype='object')


In [16]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from lightgbm import LGBMRegressor

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib

cat_cols=train.select_dtypes(include='object').columns.tolist()
cat_cols

# Define the pipeline components
weight_capacity_pipe = Pipeline(steps=[('scaler', StandardScaler())])

# Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('weight_capacity_pipe', weight_capacity_pipe, ['Weight Capacity (kg)']),  # Scaling numeric column
        ('cat_pipeline', Pipeline(steps=[  # One-Hot Encoding categorical features
            ('encoder', OneHotEncoder())
        ]), cat_cols)
    ],
    remainder='passthrough'
)

# Save the preprocessor pipeline to a .pkl file
joblib.dump(preprocessor, 'preprocessor_pipeline.pkl')
print("📂 Preprocessor pipeline saved to 'preprocessor_pipeline.pkl'.")


📂 Preprocessor pipeline saved to 'preprocessor_pipeline.pkl'.


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(train.drop(columns='Price'),train['Price'],test_size=0.2)

In [19]:
# import joblib
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error
# import numpy as np
# import pandas as pd
# from sklearn.linear_model import LinearRegression  # Import Linear Regression

# # Ensure preprocessor is defined (Replace with actual preprocessing steps)
# assert 'preprocessor' in globals(), "Define 'preprocessor' before using it in the pipeline."

# # Initialize KFold
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# # DataFrame to store out-of-fold predictions
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_LR', 'Fold'])

# # Ensure X_train and y_train are defined before using them
# assert 'X_train' in globals() and 'y_train' in globals(), "Ensure X_train and y_train are defined."

# # Convert X_train and y_train to DataFrames if they aren't already
# X_train = pd.DataFrame(X_train)
# y_train = pd.Series(y_train, name='Price')  # Ensure y_train is explicitly set to 'Price'

# # Initialize array to store out-of-fold predictions
# oof_preds = np.zeros(len(X_train))

# # K-Fold Cross-Validation Loop
# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
#     print(f"\n🔄 Training Fold {fold}/{n_folds}...")

#     # Split data into training and validation sets
#     X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#     # Create pipeline with preprocessor and Linear Regression model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),  # Replace with actual preprocessor
#         ('model', LinearRegression())  # Linear Regression model
#     ])

#     # Train the model
#     pipeline.fit(X_tr, y_tr)

#     # Save model for the current fold
#     joblib.dump(pipeline, f'linear_regression_fold_{fold}.pkl')
#     print(f"💾 Model for Fold {fold} saved as 'linear_regression_fold_{fold}.pkl'")

#     # Predict on validation set (OOF predictions)
#     y_val_pred = pipeline.predict(X_val)
#     oof_preds[val_idx] = y_val_pred  # Store OOF predictions

#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"✅ Fold {fold} RMSE: {fold_rmse:.4f}")

#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X_train.index[val_idx],  # Assuming index represents unique IDs
#         'Actual': y_val.values,
#         'OOF_Pred_LR': y_val_pred,
#         'Fold': fold
#     })

#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall OOF RMSE
# oof_rmse = mean_squared_error(y_train, oof_preds, squared=False)
# print(f"\n🏆 Overall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions to CSV
# oof_df.to_csv('oof_predictions_lr.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_lr.csv'.")

# # Train final model on the entire dataset
# final_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),  # Replace with actual preprocessor
#     ('model', LinearRegression())  # Linear Regression model
# ])
# final_pipeline.fit(X_train, y_train)

# # Save final model
# joblib.dump(final_pipeline, 'linear_regression_final.pkl')
# print("💾 Final model saved as 'linear_regression_final.pkl'.")

# # Display first few rows of OOF predictions
# oof_df.reset_index(drop=True, inplace=True)
# print(oof_df.head())


In [20]:
# # Make predictions on the test set
# predictions = pipeline.predict(test)

# # Create the submission DataFrame
# submission = pd.DataFrame({
#     'id': test['id'],         # Ensure 'id' exists in the test set
#     'Price': predictions      # Use predictions on the test set
# })

# # Save the DataFrame to a CSV file
# submission.to_csv('submission200.csv', index=False)

In [21]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import KFold, GridSearchCV
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import Ridge
# import numpy as np
# import pandas as pd

# # Ensure preprocessor is defined (Replace with actual preprocessing steps)
# assert 'preprocessor' in globals(), "Define 'preprocessor' before using it in the pipeline."

# # Define columns
# numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X_train.select_dtypes(include=['object']).columns

# # Create numeric transformer
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean'))  # Impute missing values with mean for numeric features
# ])

# # Create categorical transformer
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent for categorical features
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding for categorical features
# ])

# # Combine both transformations using ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ])

# # Initialize KFold
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# # DataFrame to store out-of-fold predictions
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_Ridge', 'Fold'])

# # Convert X_train and y_train to DataFrames if they aren't already
# X_train = pd.DataFrame(X_train)
# y_train = pd.Series(y_train)

# # Initialize array to store out-of-fold predictions
# oof_preds = np.zeros(len(X_train))

# # Ridge hyperparameter grid for tuning alpha
# param_grid = {
#     'model__alpha': [0.01, 0.1, 1, 10, 100]  # Regularization strength (alpha)
# }

# # K-Fold Cross-Validation Loop
# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
#     print(f"\n🔄 Training Fold {fold}/{n_folds}...")

#     # Split data into training and validation sets
#     X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#     # Create pipeline with imputation, preprocessor, and Ridge regression model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),  # Apply transformations to numeric and categorical features
#         ('model', Ridge())  # Use Ridge regression instead of Linear Regression
#     ])

#     # Set up GridSearchCV for hyperparameter tuning
#     grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

#     # Train the model using grid search
#     grid_search.fit(X_tr, y_tr)

#     # Best model after tuning
#     best_model = grid_search.best_estimator_

#     # Predict on validation set (OOF predictions) using the best model
#     y_val_pred = best_model.predict(X_val)
#     oof_preds[val_idx] = y_val_pred  # Store OOF predictions

#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"✅ Fold {fold} RMSE: {fold_rmse:.4f}")

#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X_train.index[val_idx],  # Assuming index represents unique IDs
#         'Actual': y_val.values,
#         'OOF_Pred_Ridge': y_val_pred,
#         'Fold': fold
#     })

#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall OOF RMSE
# oof_rmse = mean_squared_error(y_train, oof_preds, squared=False)
# print(f"\n🏆 Overall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions to CSV
# oof_df.to_csv('oof_predictions_ridge.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_ridge.csv'.")

# # Display first few rows of OOF predictions
# oof_df.reset_index(drop=True, inplace=True)
# print(oof_df.head())


In [22]:
# import joblib

# # Save the best model to a .pkl file
# joblib.dump(best_model, 'ridge_regression_model.pkl')
# print("✅ Model saved as 'ridge_regression_model.pkl'")


In [23]:
# # After training and obtaining out-of-fold predictions (as you're already doing)
# # Make predictions on the test set (Replace 'test' with your actual test DataFrame)
# test = pd.DataFrame(test)  # Ensure test is a DataFrame
# assert 'id' in test.columns, "The test DataFrame must contain an 'id' column."

# # Make predictions on the test set
# test_predictions = best_model.predict(test)  # Make predictions using the best model after GridSearchCV

# # Create the submission DataFrame
# submission = pd.DataFrame({
#     'id': test['id'],         # Ensure 'id' exists in the test set
#     'Price': test_predictions  # Use predictions on the test set
# })

# # Save the DataFrame to a CSV file
# submission.to_csv('submissiono0123.csv', index=False)

# # Save the OOF predictions DataFrame to a CSV file
# oof_df.to_csv('oof_predictions_ridge.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_ridge.csv'.")
